In [1]:
#read in cassis 
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator, List

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

from allennlp.predictors.predictor import Predictor
import spacy

PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"

BERT_PATH=os.path.join(  PATH, "tests/test_files/models", "bert_model", "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH, "tests/test_files/models", "spacy_model", "spacy-textcat" )

TEMPLATE_PATH=os.path.join( PATH, "tests/test_files/templates/out.html.template" )

sys.path.append( os.path.join( PATH ) )

with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/double_nested_list_response.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
    response = json.load(json_file)  
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/minus_lesser_of.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem  )


In [2]:
from src.transform import ListTransformer
from src.reporting_obligations import ReportingObligationsFinder

from src.utils import SeekableIterator

#from src.transform import get_other_lines, transform_lines, flatten_offsets

print( f"loading AllenNLP predictor from {BERT_PATH}" )
bert_model = Predictor.from_path( BERT_PATH )

print( f"loading spacy model from {SPACY_PATH}" )
nlp=spacy.load( SPACY_PATH )

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat


In [3]:
transformer = ListTransformer( cas )
transformer.add_list_view( OldSofaID )
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )

Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/minus_lesser_of.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template


In [9]:
print( reporting_obligations_finder.cas.get_view( 'html2textView' ).sofa_string[ 2292:11281 ] )

1.   Institutions shall report to the competent authorities, in accordance with the reporting requirements set out in Article 415(1) and the uniform reporting formats referred to in Article 415(3), the following items and their components in order to allow an assessment of the availability of stable funding:
   
      
      
      
         
            
               (a)
            
            
               the following own funds, after deductions have been applied, where appropriate:
               
                  
                  
                  
                     
                        
                           (i)
                        
                        
                           tier 1 capital instruments;
                        
                     
                  
               
               
                  
                  
                  
                     
                        
                           (ii)
            

In [5]:
for item in list_xml:
    print(item.lastChild.toxml())

AttributeError: 'NoneType' object has no attribute 'lastChild'

In [9]:
#item=list_xml[0]
offset=10
for i in range( len( list_xml ) ):
    list_xml[i].lastChild.setAttribute( 'offset_original_document', str(offset))

In [12]:
list_xml[2].lastChild.toxml()

'<h6 offset_original_document="10">Article 4</h6>'

In [6]:
#item=list_xml[0]
offset=10
for item in list_xml:
    item.lastChild.setAttribute( 'offset_original_document', str(offset))

In [8]:
list_xml[0].lastChild.toxml()

'<p data-frequency-class="2" data-frequency-could-be-1="false" data-frequency-could-be-2="true" data-frequency-could-be-3="false" data-frequency-might-be-1="true" data-frequency-might-be-2="true" data-frequency-might-be-3="false" data-frequency-split="0.20515594523894834|0.7843106687068939|0.01053336447566835" offset_original_document="10">Whereas , , <span class="ARG0">Such a final system</span> <span class="ARGM-MOD">should</span> <span class="V">provide</span> <span class="ARG1">for the determination and raising of administrative contributions</span> <span class="ARGM-PRP">not only for significant entities , but for all entities that are subjected to the Single Resolution Mechanism</span> . The contributions should be calculated at the highest level of consolidation of those entities within the participating Member States , since all subsidiaries of a group inside the perimeter of consolidation fall within the scope of the decision - making powers of the Board when adopting group re

In [15]:
test=[[1],[2],[4]]
for item in test:
    item.append(2 )
test

[[1, 2], [2, 2], [4, 2]]

In [39]:
import copy
for item in list_xml:
    #item2=copy.deepcopy( item )
    print(item.lastChild.setAttribute( 'offset_original_document', str(offset))   )
    

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [49]:
item=list_xml[0]
list_xml[0].lastChild.setAttribute( 'offset_original_document', str(offset))

In [51]:
list_xml[0].lastChild.toxml()

'<p data-frequency-class="2" data-frequency-could-be-1="false" data-frequency-could-be-2="true" data-frequency-could-be-3="false" data-frequency-might-be-1="true" data-frequency-might-be-2="true" data-frequency-might-be-3="false" data-frequency-split="0.20515594523894834|0.7843106687068939|0.01053336447566835" offset_original_document="(1, 2)">Whereas , , <span class="ARG0">Such a final system</span> <span class="ARGM-MOD">should</span> <span class="V">provide</span> <span class="ARG1">for the determination and raising of administrative contributions</span> <span class="ARGM-PRP">not only for significant entities , but for all entities that are subjected to the Single Resolution Mechanism</span> . The contributions should be calculated at the highest level of consolidation of those entities within the participating Member States , since all subsidiaries of a group inside the perimeter of consolidation fall within the scope of the decision - making powers of the Board when adopting grou

In [36]:
item

In [40]:
list_xml[0].lastChild.setAttribute( 'offset_original_document', str(offset))

In [41]:
list_xml

In [22]:
list_xml[0].lastChild.setAttribute( 'offset_original_document', str(offset)) 

In [14]:
item

In [16]:
test=list_xml[0]

In [17]:
test.lastChild.setAttribute( 'offset_original_document', str(offset)) 

In [18]:
test

In [6]:
offset=(1,2)

In [7]:
test_list=[xml_item.lastChild.setAttribute( 'offset_original_document', str(offset)) for xml_item in list_xml]

In [6]:
list_xml[0]

In [8]:
offset=(1,2)
xml_item=list_xml[0]
xml_item.lastChild.setAttribute( 'offset_original_document', str(offset))

In [10]:
xml_item

In [ ]:
offset=

list_xml_sentence=[xml_item.lastChild.setAttribute( 'offset_original_document', str(offset)) for xml_item in list_xml]


In [5]:
list_xml

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [66]:
list_xml[0].lastChild.setAttribute( 'offset_original_document', "(12,10)"   )

In [68]:
list_xml[0].lastChild.toxml()

'<p data-frequency-class="2" data-frequency-could-be-1="false" data-frequency-could-be-2="true" data-frequency-could-be-3="false" data-frequency-might-be-1="true" data-frequency-might-be-2="true" data-frequency-might-be-3="false" data-frequency-split="0.20515594523894834|0.7843106687068939|0.01053336447566835" offset_original_document="(12,10)">Whereas , , <span class="ARG0">Such a final system</span> <span class="ARGM-MOD">should</span> <span class="V">provide</span> <span class="ARG1">for the determination and raising of administrative contributions</span> <span class="ARGM-PRP">not only for significant entities , but for all entities that are subjected to the Single Resolution Mechanism</span> . The contributions should be calculated at the highest level of consolidation of those entities within the participating Member States , since all subsidiaries of a group inside the perimeter of consolidation fall within the scope of the decision - making powers of the Board when adopting gro

In [63]:
string_xml=list_xml[0].lastChild.toxml()
string_xml

'<p data-frequency-class="2" data-frequency-could-be-1="false" data-frequency-could-be-2="true" data-frequency-could-be-3="false" data-frequency-might-be-1="true" data-frequency-might-be-2="true" data-frequency-might-be-3="false" data-frequency-split="0.20515594523894834|0.7843106687068939|0.01053336447566835">Whereas , , <span class="ARG0">Such a final system</span> <span class="ARGM-MOD">should</span> <span class="V">provide</span> <span class="ARG1">for the determination and raising of administrative contributions</span> <span class="ARGM-PRP">not only for significant entities , but for all entities that are subjected to the Single Resolution Mechanism</span> . The contributions should be calculated at the highest level of consolidation of those entities within the participating Member States , since all subsidiaries of a group inside the perimeter of consolidation fall within the scope of the decision - making powers of the Board when adopting group resolution plans , carrying out 

In [64]:
#from xml.dom.minidom import parseString
#test_xml=parseString( string_xml )

In [65]:
#test_xml.lastChild.toxml()

'<p data-frequency-class="2" data-frequency-could-be-1="false" data-frequency-could-be-2="true" data-frequency-could-be-3="false" data-frequency-might-be-1="true" data-frequency-might-be-2="true" data-frequency-might-be-3="false" data-frequency-split="0.20515594523894834|0.7843106687068939|0.01053336447566835">Whereas , , <span class="ARG0">Such a final system</span> <span class="ARGM-MOD">should</span> <span class="V">provide</span> <span class="ARG1">for the determination and raising of administrative contributions</span> <span class="ARGM-PRP">not only for significant entities , but for all entities that are subjected to the Single Resolution Mechanism</span> . The contributions should be calculated at the highest level of consolidation of those entities within the participating Member States , since all subsidiaries of a group inside the perimeter of consolidation fall within the scope of the decision - making powers of the Board when adopting group resolution plans , carrying out 

In [57]:
list_xml[0].lastChild.setAttribute( 'offset_original_document', "(12,10)"   )

In [58]:
list_xml[0].lastChild.toxml()

'<p data-frequency-class="2" data-frequency-could-be-1="false" data-frequency-could-be-2="true" data-frequency-could-be-3="false" data-frequency-might-be-1="true" data-frequency-might-be-2="true" data-frequency-might-be-3="false" data-frequency-split="0.20515594523894834|0.7843106687068939|0.01053336447566835" offset="0" offset_original_document="(12,10)">Whereas , , <span class="ARG0">Such a final system</span> <span class="ARGM-MOD">should</span> <span class="V">provide</span> <span class="ARG1">for the determination and raising of administrative contributions</span> <span class="ARGM-PRP">not only for significant entities , but for all entities that are subjected to the Single Resolution Mechanism</span> . The contributions should be calculated at the highest level of consolidation of those entities within the participating Member States , since all subsidiaries of a group inside the perimeter of consolidation fall within the scope of the decision - making powers of the Board when a

In [ ]:
srl_dom_output.lastChild.setAttribute('data-frequency-could-be-2', 'true' if cat2_pred > 0.35 else 'false')


In [21]:
from xml.etree import ElementTree

xml = ElementTree.Element("Person", Name="John")
xml_str = ElementTree.tostring(xml).decode()
print(xml_str)

<Person Name="John" />


In [22]:
xml

<Element 'Person' at 0x7f0cdadcf548>

In [23]:
list_xml

In [4]:
offsets=[]
sentences=[]
for sentence in transformer.cas.get_view( NewSofaID ).sofa_string.split( "\n" ):
    offset=eval(sentence.split( "|" )[-1])
    assert type( offset ) ==tuple
    offsets.append( offset )
    sentences.append( "|".join(sentence.split( "|" )[:-1]) )

assert( len( offsets ) == len( sentences ) )

In [59]:
offsets

[(184, 197),
 (242, 244),
 (289, 327),
 (372, 379),
 (427, 473),
 (478, 498),
 (503, 605),
 (610, 634),
 (639, 708),
 (713, 1108),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (10256, 10284),
 (10289, 10298),
 (10303, 10317),
 (10322, 11463),
 (10322, 11463),
 (10322, 11463),
 (10322, 11463),
 (11506, 11515),
 (11520, 11531),
 (11536, 12975),
 (13018, 13027),
 (13032, 13098),
 (13103, 13515),
 (13520, 13641),
 (13646, 13655),
 (13660, 13710),
 (13715, 15665),
 (15708, 15957),
 (15962, 16112),
 (16117, 16323),
 (16328, 16337),
 (16342, 16392),
 (16397, 16643),
 (16648, 16865),
 (16870, 18121),
 (16870, 18121),
 (16870, 18121),
 (16870, 18121),
 (16870, 18121),
 (18164, 18349),
 (18354, 18789),
 (18794, 18963),
 (18968, 18977),
 (18982, 19051),
 (19056, 19481),
 (19486, 21280),
 

In [60]:
sentences

['19.12.2017',
 'EN',
 'Official Journal of the European Union',
 'L 337/6',
 'COMMISSION DELEGATED REGULATION (EU) 2017/2361',
 'of 14 September 2017',
 'on the final system of contributions to the administrative expenditures of the Single Resolution Board',
 'THE EUROPEAN COMMISSION,',
 'Having regard to the Treaty on the Functioning of the European Union,',
 'Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,',
 'Whereas, , The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down in Commission Delegated Regulation (EU) No 1310/2014 (2), was based on a simplified methodology and covered on

In [52]:
isinstance( eval( offset ) , tuple )

True

In [41]:
offsets[0].split( "," )[0].strip(" (")

'184'

In [42]:
offsets[0].split( "," )[1].strip(" )")

'197'

In [48]:
eval( offsets[0] )[0]

184

In [28]:
test=""
"|".join( test.split( "|" )[:-1])

''

In [18]:
test="some| some|(12,22)"
"|".join( test.split( "|" )[:-1])

'some| some'

In [22]:
test="|some| some|(12,22)"
"|".join( test.split( "|" )[:-1])

'|some| some'

In [8]:
sentence.split( "|" )[-1] for sentence in transformer.cas.get_view( NewSofaID ).sofa_string.split( "\n" )

['19.12.2017|(184, 197)',
 'EN|(242, 244)',
 'Official Journal of the European Union|(289, 327)',
 'L 337/6|(372, 379)',
 'COMMISSION DELEGATED REGULATION (EU) 2017/2361|(427, 473)',
 'of 14 September 2017|(478, 498)',
 'on the final system of contributions to the administrative expenditures of the Single Resolution Board|(503, 605)',
 'THE EUROPEAN COMMISSION,|(610, 634)',
 'Having regard to the Treaty on the Functioning of the European Union,|(639, 708)',
 'Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,|(713, 1108)',
 'Whereas, , The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down

In [ ]:
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
#reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )

In [5]:
path="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/sofa_listview/"
with open( os.path.join( path, "minus_lesser_of.txt"  ) , "w") as f: 
    f.write(  transformer.cas.get_view( NewSofaID ).sofa_string ) 
    

In [11]:
path="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/sofa_listview/"
test=open( os.path.join(path, "minus_lesser_of.txt") ).read()

In [13]:
transformer.cas.get_view( NewSofaID ).sofa_string == test

True

In [3]:
value_between_tagtype_generator=cas.get_view( OldSofaID ).select( value_between_tagtype )     

seek_vbtt=SeekableIterator( iter(value_between_tagtype_generator) )

lines, offsets = get_other_lines( cas , OldSofaID, seek_vbtt, 'root', paragraph_type=paragraph_type )

flatten_offsets( offsets )

assert len( lines ) == len( offsets )

transformed_lines, transformed_lines_offsets = transform_lines( lines, offsets )

In [4]:
import pickle
path="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/pickle"
with open( os.path.join( path, "minus_lesser_of_offsets.p" ) , 'wb') as f:
    pickle.dump(offsets, f)

In [5]:
len( lines)

19

In [8]:
len( offsets )

19

In [ ]:
flatten_offsets( offsets )

assert len( lines ) == len( offsets )

transformed_lines, transformed_lines_offsets =transform_lines( lines, offsets )

assert len( transformed_lines ) == len( transformed_lines_offsets )

lines_offsets=[]
for line, offset in zip( transformed_lines, transformed_lines_offsets  ):
    lines_offsets.append(line + "|" + str( offset ))

In [8]:
lines

['19.12.2017   ',
 'EN',
 'Official Journal of the European Union',
 'L 337/6',
 'COMMISSION DELEGATED REGULATION (EU) 2017/2361',
 'of 14 September 2017',
 'on the final system of contributions to the administrative expenditures of the Single Resolution Board',
 'THE EUROPEAN COMMISSION,',
 'Having regard to the Treaty on the Functioning of the European Union,',
 'Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,',
 ['Whereas:',
  '(1)',
  'The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down in Commission Delegated Regulation (EU) No 1310/2014 (2), was based on a simplified methodolog

In [9]:
import pickle
with open( "/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/pickle/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_nested_lines.p"  , 'rb') as f:
    # read the data as binary data stream
    lines_ok = pickle.load(f)

In [3]:
#ListTransformer( cas ).add_list_view( OldSofaID )

In [6]:
#print( cas.get_view( "ListView" ).sofa_string )

19.12.2017|(184, 197)
EN|(242, 244)
Official Journal of the European Union|(289, 327)
L 337/6|(372, 379)
COMMISSION DELEGATED REGULATION (EU) 2017/2361|(427, 473)
of 14 September 2017|(478, 498)
on the final system of contributions to the administrative expenditures of the Single Resolution Board|(503, 605)
THE EUROPEAN COMMISSION,|(610, 634)
Having regard to the Treaty on the Functioning of the European Union,|(639, 708)
Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,|(713, 1108)
Whereas, , The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down in Commission Delegated Regulation (EU) N

In [27]:
line_offset=[]
for line, offset in zip( transformed_lines, transformed_lines_offsets  ):
    line_offset.append(line + "|" + str( offset ))

In [28]:
line_offset

['19.12.2017|(184, 197)',
 'EN|(242, 244)',
 'Official Journal of the European Union|(289, 327)',
 'L 337/6|(372, 379)',
 'COMMISSION DELEGATED REGULATION (EU) 2017/2361|(427, 473)',
 'of 14 September 2017|(478, 498)',
 'on the final system of contributions to the administrative expenditures of the Single Resolution Board|(503, 605)',
 'THE EUROPEAN COMMISSION,|(610, 634)',
 'Having regard to the Treaty on the Functioning of the European Union,|(639, 708)',
 'Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,|(713, 1108)',
 'Whereas, , The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down

In [ ]:
cas.get_view( NewSofaID).sofa_string = "\n".join( transformed_lines )

In [6]:
transformed_lines

['19.12.2017',
 'EN',
 'Official Journal of the European Union',
 'L 337/6',
 'COMMISSION DELEGATED REGULATION (EU) 2017/2361',
 'of 14 September 2017',
 'on the final system of contributions to the administrative expenditures of the Single Resolution Board',
 'THE EUROPEAN COMMISSION,',
 'Having regard to the Treaty on the Functioning of the European Union,',
 'Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,',
 'Whereas, , The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down in Commission Delegated Regulation (EU) No 1310/2014 (2), was based on a simplified methodology and covered on

In [7]:
transformed_lines_offsets

[(184, 197),
 (242, 244),
 (289, 327),
 (372, 379),
 (427, 473),
 (478, 498),
 (503, 605),
 (610, 634),
 (639, 708),
 (713, 1108),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (1113, 10213),
 (10256, 10284),
 (10289, 10298),
 (10303, 10317),
 (10322, 11463),
 (10322, 11463),
 (10322, 11463),
 (10322, 11463),
 (11506, 11515),
 (11520, 11531),
 (11536, 12975),
 (13018, 13027),
 (13032, 13098),
 (13103, 13515),
 (13520, 13641),
 (13646, 13655),
 (13660, 13710),
 (13715, 15665),
 (15708, 15957),
 (15962, 16112),
 (16117, 16323),
 (16328, 16337),
 (16342, 16392),
 (16397, 16643),
 (16648, 16865),
 (16870, 18121),
 (16870, 18121),
 (16870, 18121),
 (16870, 18121),
 (16870, 18121),
 (18164, 18349),
 (18354, 18789),
 (18794, 18963),
 (18968, 18977),
 (18982, 19051),
 (19056, 19481),
 (19486, 21280),
 

In [23]:
sentence_offset = 'test and | test'+ '|' +str( transformed_lines_offsets[2] )

In [24]:
sentence_offset.split( "|" )[-1]

'(289, 327)'

In [6]:
assert( len(  transformed_lines ) ) == len( transformed_lines_offsets )

In [11]:
transformed_lines

['19.12.2017',
 'EN',
 'Official Journal of the European Union',
 'L 337/6',
 'COMMISSION DELEGATED REGULATION (EU) 2017/2361',
 'of 14 September 2017',
 'on the final system of contributions to the administrative expenditures of the Single Resolution Board',
 'THE EUROPEAN COMMISSION,',
 'Having regard to the Treaty on the Functioning of the European Union,',
 'Having regard to Regulation (EU) No 806/2014 of the European Parliament and of the Council of 15 July 2014 establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 (1), and in particular Article 65(5) thereof,',
 'Whereas, , The provisional system of instalments on contributions to the administrative expenditures of the Board, laid down in Commission Delegated Regulation (EU) No 1310/2014 (2), was based on a simplified methodology and covered on

In [43]:
transformed_lines_offsets[-29:]

[(35443, 37486),
 (35443, 37486),
 (35443, 37486),
 (35443, 37486),
 (35443, 37486),
 (35443, 37486),
 (37529, 37972),
 (37977, 38191),
 (38196, 38206),
 (38211, 38256),
 (38261, 38452),
 (38457, 38467),
 (38472, 38478),
 (38483, 38534),
 (38539, 38549),
 (38554, 38570),
 (38575, 38715),
 (38728, 38822),
 (38830, 38866),
 (38885, 38928),
 (38939, 38977),
 (38988, 39007),
 (39030, 39069),
 (39074, 39344),
 (39349, 39503),
 (39508, 39744),
 (39749, 39885),
 (39890, 40181),
 (40186, 40535)]

In [47]:
transformed_lines[-29:]

["6.   Where the Board fully or partially outsources tasks in accordance with paragraph 1, it shall ensure at all times that any contract concluded for the purposes of the outsourcing does not provide for the delegation of the Board's responsibility.",
 "6.   Where the Board fully or partially outsources tasks in accordance with paragraph 1, it shall ensure at all times that any contract concluded for the purposes of the outsourcing does not provide for an exclusion of the Board's accountability under Article 45 and Article 46(1) of Regulation (EU) No 806/2014, or of its independence under Article 47 of that Regulation.",
 '6.   Where the Board fully or partially outsources tasks in accordance with paragraph 1, it shall ensure at all times that the outsourcing does not result in depriving the Board from the necessary systems and controls to manage the risks it faces.',
 '6.   Where the Board fully or partially outsources tasks in accordance with paragraph 1, it shall ensure at all time

In [28]:
transformed_lines[25]

'Whereas, , Since this Regulation introduces the final system of contributions to the administrative expenditures of the Board, the Regulation on the provisional system is no longer necessary and should be repealed.'

In [48]:
print( cas.get_view( 'html2textView' ).sofa_string[ 35443:37486 ])

6.   Where the Board fully or partially outsources tasks in accordance with paragraph 1, it shall ensure at all times that:
   
      
      
      
         
            
               (a)
            
            
               any contract concluded for the purposes of the outsourcing does not provide for the delegation of the Board's responsibility;
            
         
      
   
   
      
      
      
         
            
               (b)
            
            
               any contract concluded for the purposes of the outsourcing does not provide for an exclusion of the Board's accountability under Article 45 and Article 46(1) of Regulation (EU) No 806/2014, or of its independence under Article 47 of that Regulation;
            
         
      
   
   
      
      
      
         
            
               (c)
            
            
               the outsourcing does not result in depriving the Board from the necessary systems and controls to manage the

In [20]:
print( cas.get_view( 'html2textView' ).sofa_string[ 11401:11462 ] )

Treatment of maturity mismatches in synthetic securitisations


In [19]:
transformed_lines=transform_lines( lines )

In [22]:
len( transformed_lines )

19

In [23]:
transformed_lines

['5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.',
 '6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.',
 'EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.',
 'Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.',
 '7.   Institutions shall not report inflows 

In [27]:
[( 1,2 )]*5

[(1, 2), (1, 2), (1, 2), (1, 2), (1, 2)]

In [25]:
type( [ 'f', 1 , 1 ] )

list

['5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.',
 '6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.',
 'EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.',
 'Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.',
 '7.   Institutions shall not report inflows 

In [12]:
offsets

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 (2292, 11281),
 (11401, 11462),
 (11466, 14783),
 (14903, 14927)]

In [13]:
# (2292, 11281)
print( cas.get_view( 'html2textView' ).sofa_string[ 2292:11281 ])

1.   Institutions shall report to the competent authorities, in accordance with the reporting requirements set out in Article 415(1) and the uniform reporting formats referred to in Article 415(3), the following items and their components in order to allow an assessment of the availability of stable funding:
   
      
      
      
         
            
               (a)
            
            
               the following own funds, after deductions have been applied, where appropriate:
               
                  
                  
                  
                     
                        
                           (i)
                        
                        
                           tier 1 capital instruments;
                        
                     
                  
               
               
                  
                  
                  
                     
                        
                           (ii)
            

In [14]:
lines[15]

['1.   Institutions shall report to the competent authorities, in accordance with the reporting requirements set out in Article 415(1) and the uniform reporting formats referred to in Article 415(3), the following items and their components in order to allow an assessment of the availability of stable funding:',
 '(a)',
 ['the following own funds, after deductions have been applied, where appropriate:',
  '(i)',
  'tier 1 capital instruments;',
  '(ii)',
  'tier 2 capital instruments;',
  '(iii)',
  'other preferred shares and capital instruments in excess of Tier 2 allowable amount having an effective maturity of one year or greater;'],
 '(b)',
 ['the following liabilities not included in point (a):',
  '(i)',
  'retail deposits that qualify for the treatment set out in Article 421(1);',
  '(ii)',
  'retail deposits that qualify for the treatment set out in Article 421(2);',
  '(iii)',
  'deposits that qualify for the treatment set out in Article 422 (3) and (4);',
  '(iv)',
  'of the

In [6]:
lines

['5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.',
 '6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.',
 'EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.',
 'Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.',
 '7.   Institutions shall not report inflows 

In [5]:
lines

['5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.',
 '6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.',
 'EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.',
 'Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.',
 '7.   Institutions shall not report inflows 

In [6]:
offsets

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 [(2292, 2601),
  (2665, 2668),
  [(2710, 2789),
   (2937, 2940),
   (3018, 3045),
   (3275, 3279),
   (3357, 3384),
   (3614, 3619),
   (3697, 3833)],
  (4013, 4016),
  [(4058, 4110),
   (4258, 4261),
   (4339, 4412),
   (4642, 4646),
   (4724, 4797),
   (5027, 5032),
   (5110, 5185),
   (5415, 5419),
   (5497, 5747),
   (5977, 5980),
   (6058, 6152),
   (6382, 6386),
   (6464, 6558),
   (6788, 6793),
   (6871, 6981),
   (7211, 7217),
   (7295, 7341),
   (7571, 7575),
   [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
   (9146, 9149),
   (9227, 9393),
   (9623, 9627),
   [(9705, 9803),
    (10035, 10036),
    (10150, 10245),
    (10607, 10608),
    (10722, 10816)],
   (11176, 11181),
   (11259, 11281)]],
 (11401, 11462),
 [(11466, 11762),
  (11826, 11

In [8]:
flatten_offsets( offsets  )

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 (2292, 11281),
 (11401, 11462),
 (11466, 14783),
 (14903, 14927)]

In [9]:
offsets

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 (2292, 11281),
 (11401, 11462),
 (11466, 14783),
 (14903, 14927)]

In [6]:
def flatten(container):
    for i in container:
        if isinstance(i, (list)):
            for j in flatten(i):
                yield j
        else:
            yield i

In [7]:
for i,item in enumerate(offsets):
    if type(item)==list:
        flattened_list=list( flatten( item ) )
        if flattened_list:
            offsets[i]=( flattened_list[0][0], flattened_list[-1][-1] )
        
#list( flatten( offsets[15] ) )

In [8]:
offsets

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 (2292, 11281),
 (11401, 11462),
 (11466, 14783),
 (14903, 14927)]

In [8]:
len( lines )

19

In [9]:
len( offsets )

19

In [38]:
len( offsets )

19

In [43]:
offsets[15][0][0]

2292

In [44]:
offsets[15][-1][-1]

11281

In [46]:
offsets[15]

[(2292, 2601),
 (2665, 2668),
 (2710, 2789),
 (2937, 2940),
 (3018, 3045),
 (3275, 3279),
 (3357, 3384),
 (3614, 3619),
 (3697, 3833),
 (4013, 4016),
 (4058, 4110),
 (4258, 4261),
 (4339, 4412),
 (4642, 4646),
 (4724, 4797),
 (5027, 5032),
 (5110, 5185),
 (5415, 5419),
 (5497, 5747),
 (5977, 5980),
 (6058, 6152),
 (6382, 6386),
 (6464, 6558),
 (6788, 6793),
 (6871, 6981),
 (7211, 7217),
 (7295, 7341),
 (7571, 7575),
 (7653, 7835),
 (8067, 8068),
 (8182, 8274),
 (8636, 8637),
 (8751, 8786),
 (9146, 9149),
 (9227, 9393),
 (9623, 9627),
 (9705, 9803),
 (10035, 10036),
 (10150, 10245),
 (10607, 10608),
 (10722, 10816),
 (11176, 11181),
 (11259, 11281)]

In [13]:
len(lines)

19

In [21]:
offsets

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 [(2292, 2601),
  (2665, 2668),
  [(2710, 2789),
   (2937, 2940),
   (3018, 3045),
   (3275, 3279),
   (3357, 3384),
   (3614, 3619),
   (3697, 3833)],
  (4013, 4016),
  [(4058, 4110),
   (4258, 4261),
   (4339, 4412),
   (4642, 4646),
   (4724, 4797),
   (5027, 5032),
   (5110, 5185),
   (5415, 5419),
   (5497, 5747),
   (5977, 5980),
   (6058, 6152),
   (6382, 6386),
   (6464, 6558),
   (6788, 6793),
   (6871, 6981),
   (7211, 7217),
   (7295, 7341),
   (7571, 7575),
   [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
   (9146, 9149),
   (9227, 9393),
   (9623, 9627),
   [(9705, 9803),
    (10035, 10036),
    (10150, 10245),
    (10607, 10608),
    (10722, 10816)],
   (11176, 11181),
   (11259, 11281)]],
 (11401, 11462),
 [(11466, 11762),
  (11826, 11

In [10]:
offsets[15]  #flatten and take begin and end..

[(2292, 2601),
 (2665, 2668),
 [(2710, 2789),
  (2937, 2940),
  (3018, 3045),
  (3275, 3279),
  (3357, 3384),
  (3614, 3619),
  (3697, 3833)],
 (4013, 4016),
 [(4058, 4110),
  (4258, 4261),
  (4339, 4412),
  (4642, 4646),
  (4724, 4797),
  (5027, 5032),
  (5110, 5185),
  (5415, 5419),
  (5497, 5747),
  (5977, 5980),
  (6058, 6152),
  (6382, 6386),
  (6464, 6558),
  (6788, 6793),
  (6871, 6981),
  (7211, 7217),
  (7295, 7341),
  (7571, 7575),
  [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
  (9146, 9149),
  (9227, 9393),
  (9623, 9627),
  [(9705, 9803),
   (10035, 10036),
   (10150, 10245),
   (10607, 10608),
   (10722, 10816)],
  (11176, 11181),
  (11259, 11281)]]

In [16]:
flat_list = [item for sublist in offsets[15] for item in sublist]

In [ ]:
for offset in offsets:
    if type( offset ) == list:
        

In [23]:
import numpy as np
np.array( offsets )

/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


array([(4, 515), (519, 659), (663, 759), (763, 951), (955, 1162),
       (1166, 1247), (1251, 1558), (1562, 1573), (1577, 1615),
       (1619, 2000), (2004, 2184), (2188, 2197), (2201, 2239),
       (2243, 2254), (2258, 2288),
       list([(2292, 2601), (2665, 2668), [(2710, 2789), (2937, 2940), (3018, 3045), (3275, 3279), (3357, 3384), (3614, 3619), (3697, 3833)], (4013, 4016), [(4058, 4110), (4258, 4261), (4339, 4412), (4642, 4646), (4724, 4797), (5027, 5032), (5110, 5185), (5415, 5419), (5497, 5747), (5977, 5980), (6058, 6152), (6382, 6386), (6464, 6558), (6788, 6793), (6871, 6981), (7211, 7217), (7295, 7341), (7571, 7575), [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)], (9146, 9149), (9227, 9393), (9623, 9627), [(9705, 9803), (10035, 10036), (10150, 10245), (10607, 10608), (10722, 10816)], (11176, 11181), (11259, 11281)]]),
       (11401, 11462),
       list([(11466, 11762), (11826, 11829), (11871, 12104), (12202, 12205), (12247, 12571), [(12603, 12609), (12

In [25]:
nests = [1, 2, [3, 4, [5],['hi']], [6, [[[7, 'hello']]]]]

def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

print(list(flatten(nests)))

[1, 2, 3, 4, 5, 'hi', 6, 7, 'hello']


In [19]:
type( offsets[15] ) == list

True

In [20]:
offsets[15]

[(2292, 2601),
 (2665, 2668),
 [(2710, 2789),
  (2937, 2940),
  (3018, 3045),
  (3275, 3279),
  (3357, 3384),
  (3614, 3619),
  (3697, 3833)],
 (4013, 4016),
 [(4058, 4110),
  (4258, 4261),
  (4339, 4412),
  (4642, 4646),
  (4724, 4797),
  (5027, 5032),
  (5110, 5185),
  (5415, 5419),
  (5497, 5747),
  (5977, 5980),
  (6058, 6152),
  (6382, 6386),
  (6464, 6558),
  (6788, 6793),
  (6871, 6981),
  (7211, 7217),
  (7295, 7341),
  (7571, 7575),
  [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
  (9146, 9149),
  (9227, 9393),
  (9623, 9627),
  [(9705, 9803),
   (10035, 10036),
   (10150, 10245),
   (10607, 10608),
   (10722, 10816)],
  (11176, 11181),
  (11259, 11281)]]

In [17]:
flat_list

[2292,
 2601,
 2665,
 2668,
 (2710, 2789),
 (2937, 2940),
 (3018, 3045),
 (3275, 3279),
 (3357, 3384),
 (3614, 3619),
 (3697, 3833),
 4013,
 4016,
 (4058, 4110),
 (4258, 4261),
 (4339, 4412),
 (4642, 4646),
 (4724, 4797),
 (5027, 5032),
 (5110, 5185),
 (5415, 5419),
 (5497, 5747),
 (5977, 5980),
 (6058, 6152),
 (6382, 6386),
 (6464, 6558),
 (6788, 6793),
 (6871, 6981),
 (7211, 7217),
 (7295, 7341),
 (7571, 7575),
 [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
 (9146, 9149),
 (9227, 9393),
 (9623, 9627),
 [(9705, 9803),
  (10035, 10036),
  (10150, 10245),
  (10607, 10608),
  (10722, 10816)],
 (11176, 11181),
 (11259, 11281)]

In [14]:
offsets

[(4, 515),
 (519, 659),
 (663, 759),
 (763, 951),
 (955, 1162),
 (1166, 1247),
 (1251, 1558),
 (1562, 1573),
 (1577, 1615),
 (1619, 2000),
 (2004, 2184),
 (2188, 2197),
 (2201, 2239),
 (2243, 2254),
 (2258, 2288),
 [(2292, 2601),
  (2665, 2668),
  [(2710, 2789),
   (2937, 2940),
   (3018, 3045),
   (3275, 3279),
   (3357, 3384),
   (3614, 3619),
   (3697, 3833)],
  (4013, 4016),
  [(4058, 4110),
   (4258, 4261),
   (4339, 4412),
   (4642, 4646),
   (4724, 4797),
   (5027, 5032),
   (5110, 5185),
   (5415, 5419),
   (5497, 5747),
   (5977, 5980),
   (6058, 6152),
   (6382, 6386),
   (6464, 6558),
   (6788, 6793),
   (6871, 6981),
   (7211, 7217),
   (7295, 7341),
   (7571, 7575),
   [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
   (9146, 9149),
   (9227, 9393),
   (9623, 9627),
   [(9705, 9803),
    (10035, 10036),
    (10150, 10245),
    (10607, 10608),
    (10722, 10816)],
   (11176, 11181),
   (11259, 11281)]],
 (11401, 11462),
 [(11466, 11762),
  (11826, 11

In [10]:
len( transformed_lines )

19

In [12]:
len(lines)

19

In [3]:
transformer=ListTransformer( cas )
transformer.add_list_view( OldSofaID='html2textView', NewSofaID = 'ListView' )

In [6]:
print( transformer.cas.get_view( 'ListView' ).sofa_string )

5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.
6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.
EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.
Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.
7.   Institutions shall not report inflows from any of the li

In [2]:
transformer=ListTransformer( cas )

transformer.add_list_view( OldSofaID='html2textView', NewSofaID = 'ListView' )

reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas(  TEMPLATE_PATH , ROSofaID='ReportingObligationsView' )

reporting_obligations_finder.print_to_html( TEMPLATE_PATH, OUTPUT_PATH )


loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat
Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/test_test.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template
